Tahminler gözlem benzerliğine göre yapılır. parametric bir yapısı yoktur. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
plt.rcParams["figure.figsize"] = (10,6)
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)

In [14]:
df = pd.read_csv("Hitters.csv")
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [15]:
X = df[["League", "Division", "NewLeague"]]

In [16]:
X.head()

,League,Division,NewLeague
0,A,E,A
1,N,W,N
2,A,W,A
3,N,E,N
4,N,E,N


In [21]:
df = df.dropna()

In [22]:
y = df["Salary"]

In [25]:
X = pd.get_dummies(data = X, drop_first = True)

In [26]:
X.head()

,League_N,Division_W,NewLeague_N
0,0,0,0
1,1,1,1
2,0,1,0
3,1,0,1
4,1,0,1


In [27]:
y.head()

1    475.0
2    480.0
3    500.0
4     91.5
5    750.0
Name: Salary, dtype: float64

In [32]:
X.shape

(322, 3)

In [33]:
y.shape

(263,)

In [35]:
X = X.iloc[0:263]
X.shape

(263, 3)

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 101)

In [28]:
from sklearn.neighbors import KNeighborsRegressor

In [37]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

def train_val(model, X_train, y_train, X_test, y_test):
    
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    scores = {"train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    "test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [40]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

operations = [("minmax", MinMaxScaler()), ("knn", KNeighborsRegressor())]
Pipeline(steps=operations)
pipe_model = Pipeline(steps=operations)
pipe_model.fit(X_train, y_train)

train_val(pipe_model, X_train, y_train, X_test, y_test)

,train,test
R2,-0.003782,-0.316376
mae,363.780794,391.943377
mse,214334.821282,201544.032579
rmse,462.963088,448.936557


### Elbow K value : 

In [42]:
min_max = MinMaxScaler()
X_train_scaled = min_max.fit_transform(X_train)
X_test_scaled = min_max.transform(X_test)

In [44]:
k_range = range(1,23)

list_dist = []
list_score = []
sayac = 1
for k in k_range : 
    model = KNeighborsRegressor(n_neighbors= k)
    model.fit(X_train_scaled, y_train)
    #print(sayac , "Doğruluk oranı = % ", model.score(X_train_scaled, y_train)*100)
    sayac = sayac + 1
    list_dist.append(sayac-1)
    list_score.append(round(model.score(X_train_scaled, y_train)*100,2))
list_score = pd.DataFrame(list_score)  
list_dist = pd.DataFrame(list_dist)
train_k = pd.concat([list_dist, list_score], axis=1)
train_k.columns = ["k_value_train", "score_train"]
train_k

list_dist1 = []
list_score1 = []
index = 1
for k in k_range : 
    model2 = KNeighborsRegressor(n_neighbors= k)
    model2.fit(X_train_scaled, y_train)
    #print(sayac , "Doğruluk oranı = % ", model.score(X_train_scaled, y_train)*100)
    index = index + 1
    list_dist1.append(index-1)
    list_score1.append(round(model2.score(X_test_scaled, y_test)*100,2))
list_score1 = pd.DataFrame(list_score1)  
list_dist1 = pd.DataFrame(list_dist1)
test_k = pd.concat([list_dist1, list_score1], axis=1)
test_k.columns = ["k_value_test", "score_test"]
scores = pd.concat([test_k, train_k], axis=1)
scores["result"] = scores["score_train"] - scores["score_test"]
scores

,k_value_test,score_test,k_value_train,score_train,result
0,1,-25.97,1,-17.35,8.62
1,2,-11.46,2,-9.62,1.84
2,3,-30.67,3,-8.33,22.34
3,4,-29.43,4,2.34,31.77
4,5,-31.64,5,-0.38,31.26
5,6,-33.50,6,-2.50,31.00
6,7,-38.23,7,-4.23,34.00
7,8,-42.12,8,-4.88,37.24
8,9,-39.95,9,-6.56,33.39
9,10,-38.36,10,-4.47,33.89


### GridSearchCV : 

In [47]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [48]:
operations = [("min_max", MinMaxScaler()), ("knn", KNeighborsRegressor())]
Pipeline(steps=operations)
model = Pipeline(steps=operations)

In [50]:
model.get_params()

{'memory': None,
 'steps': [('min_max', MinMaxScaler()), ('knn', KNeighborsRegressor())],
 'verbose': False,
 'min_max': MinMaxScaler(),
 'knn': KNeighborsRegressor(),
 'min_max__clip': False,
 'min_max__copy': True,
 'min_max__feature_range': (0, 1),
 'knn__algorithm': 'auto',
 'knn__leaf_size': 30,
 'knn__metric': 'minkowski',
 'knn__metric_params': None,
 'knn__n_jobs': None,
 'knn__n_neighbors': 5,
 'knn__p': 2,
 'knn__weights': 'uniform'}

In [51]:
k_values = range(1,30)
param_grid = {"knn__n_neighbors":k_values, "knn__p": [1,2], "knn__weights": ['uniform', "distance"]}
grid_model = GridSearchCV(model, param_grid, cv=10, scoring= 'neg_root_mean_squared_error')
grid_model.fit(X_train, y_train) # scale edilmemiş halini fit etmek zorundayız.

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('min_max', MinMaxScaler()),
                                       ('knn', KNeighborsRegressor())]),
             param_grid={'knn__n_neighbors': range(1, 30), 'knn__p': [1, 2],
                         'knn__weights': ['uniform', 'distance']},
             scoring='neg_root_mean_squared_error')

In [52]:
grid_model.best_params_

{'knn__n_neighbors': 29, 'knn__p': 1, 'knn__weights': 'uniform'}

In [53]:
train_val(grid_model, X_train, y_train, X_test, y_test)

,train,test
R2,0.008369,-0.031526
mae,344.435857,343.871980
mse,211740.384499,157931.979391
rmse,460.152567,397.406567
